# Formation RNAseq CEA - juin 2023

*Enseignantes: Sandrine Caburet et Claire Vandiedonck*

Session IFB : 5 CPU + 21 GB de RAM

# Part 5: Mapping quality check

  
- 0.1 - About session for IFB core cluster
- 0.2 -Parameters to be set or modified by the user
- 1 - Some checks as a precaution with ``samtools``
- 2 - Some metrics with ``samtools``
- 3 - Some metrics and graphical results with ``Qualimap``
- 4 - Summary report with ``MultiQC``
- 5 - Monitoring disk usage

<div class="alert alert-block alert-danger"> 
    <b>     🚨 🚨 🚨 💣 💣 💣 <u> WARNING WARNING WARNING </u> 💣 💣 💣 🚨 🚨 🚨    </b> <br><br>
    Now the mapping step is done, please go back to a lighter session with <b>only 5-CPU and 21 GB of RAM</b>.
</div>

---
---
## 0. Before going further
---

<div class="alert alert-block alert-danger"><b>Caution:</b> 
Before starting the analysis, save a backup copy of this notebok : in the left-hand panel, right-click on this file and select "Duplicate"<br>
You can also make backups during the analysis. Don't forget to save your notebook regularly: <kbd>Ctrl</kbd> + <kbd>S</kbd> or click on the 💾 icon.
</div>



### 0.1 - About session for IFB core cluster
---
<em>loaded JupyterLab</em> : Version 3.2.1

In [ ]:
## Code cell 1 ##

echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (5 CPU, 21GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}


In [ ]:
## Code cell 2 ##

module load samtools/1.15.1 qualimap/2.2.2b multiqc/1.13

echo "===== quality control & bam sorting ====="
samtools --version | head -n 2
echo "===== graphical quality controls ====="
qualimap --help | head -n 4
echo "===== quality reports compilation ====="
multiqc --version

--

### 0.2 - Parameters to be set or modified by the user
---

- Using a full path with a `/` at the end, **define the folder** where you want or have to work with `gohome` variable:

In [ ]:
## Code cell 3 ##

gohome="/shared/projects/2312_rnaseq_cea/"

echo "=== Home root folder is ==="
echo "${gohome}"
echo ""
echo "=== Working (personal) folder tree ==="
tree -L 2 "${gohome}$USER"
echo "=== current working directory ==="
echo "${PWD}"

- Precise the **maximum amount of CPU** (central processing units, cores) and **RAM-memory (in Bytes)** that programs can use.<a id="computressources"></a>

In [ ]:
## Code cell 4 ##

authorizedCPU=4

authorizedRAM=20000000000  # 20GB

- This notebook contains heavy running cells in <a href="#section3.2">section 3.2</a> so make sure before you launch these cells that they are adapted to your environment. <br>
For Samtools, the ``-m`` option is the **RAM memory size that will be used by each thread**! <br>
<blockquote>
        See option <code>--java-mem-size=20G</code> in Qualimap lines, <a href="#section3.2">section 3.2</a> <br>
        See options <code>--threads 4 -m 5G</code> in Samtools line, <a href="#section3.3">section 3.3.b</a>
</blockquote>

<div class="alert alert-block alert-danger">
    <b>Values set in this notebook are valid for a 5-CPU session with access to 21 GB of RAM</b>. Ideally, use 70-80% of the CPU amount your system or session has. <br>
    DO NOT ask for more RAM than you can use.
</div>

- With a `/` at the end, precise the folder where alignment ``_ALigned.SortedByCoord.out.bam`` files (produced by ``STAR``) are:

In [ ]:
## Code cell 5 ##

mappedfolder="${gohome}$USER/Results/star/"

- Reset the path to uncompressed annotation file, especially if you changed it in previous notebook.

In [ ]:
## Code cell 6 ##

gtffile="${gohome}allData/Reference/extracted/genome_annotation.gtf"

- Comment will later be included in Quality report to keep analysis informations handy. **Beware to adapt ``mycomment`` variable text** in [section 4.1.b](#multiqctextvar) before launching MultiQC report generation.

---
---
## 1 - Some checks as a precaution with <code>samtools</code>
---

### 1.1 - Available files
---

As we change session or even day, let's first check all files are there:

In [ ]:
## Code cell 7 ##

mappedfolder="${gohome}$USER/Results/star/"

echo "mappedfolder is "${mappedfolder}
echo "There are $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | wc -l) bam files in it:"
ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam

--
### 1.2 - Examining data files: are they what we expect?
---

These `.bam` files are smaller that ``.sam`` files as they are binary ones... but we can't see inside without a specific tool (``samtools view``) included in **samtools** suite (http://www.htslib.org/).

In [ ]:
## Code cell 8 ##

samtools --version | head -n 2

We list the files in the folder and ask for only the first line (``-n 1``) to get one sample:

In [ ]:
## Code cell 9 ##

abamfile=$(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | head -n 1)
echo ${abamfile}

Let's visualize the first lines of this file using [samtools `view`](http://www.htslib.org/doc/samtools-view.html).

In [ ]:
## Code cell 10 ##

samtools view ${abamfile} | head

<ul class="alert alert-block alert-info"><b> Info 💡 :</b><br>
    <li>
        For <b>SAM tools documentation</b>, see the <a href="http://www.htslib.org/doc/#manual-pages"><i>manual page</i></a></il>
        <li>
    For <b>SAM/BAM format specifications</b>: <br>see <a href="https://github.com/samtools/hts-specs"><i>the full SAM/BAM file format specification</i></a> with in particular this further link to the <i>gold-standard reference</i> of  <a href="http://samtools.github.io/hts-specs/SAMv1.pdf"><i>the canonical specifications of SAM/BAM</i></a>. For more information on BAM file format, see also <a href="https://biocorecrg.github.io/RNAseq_course_2019/alnpractical.html"><i>BAM/SAM/CRAM format</i></a> section of another training course made by Barcelona's CRG Biocore Facility members on 2019.
    </li>
    <li>
 To understand the <b>Flag</b> field, or at least know what is inside alignment quality scores: <br> The Broad Institute's tool <a href="https://broadinstitute.github.io/picard/explain-flags.html"><i>Explain SAM Flags</i></a> may be helpfull! To convert flags between numeric and textual representation, you may also use <a href="http://www.htslib.org/doc/samtools-flags.html">samtools <code>flag<code></a>.
</ul>

--
### 1.3 - Checking `.bam` files integrity
---

You may join this pipeline with your own `.bam` files now, either generated from another alignment tool or directly downloaded from network.  
Alternatively, you keep going with previous pipe and you want to check there were no major issues with the aligment tool we used (*STAR*) that could have led to incorrect outputs.  

So, to start off on the right foot, let's check that files are intact. To that purpose, **samtools** will once again help us to deal with `.bam` files using the [`quickcheck` tool](http://www.htslib.org/doc/samtools-quickcheck.html). It checks the beginng and the end of each file.

In [ ]:
## Code cell 11 ##

logfile="${mappedfolder}samtools_quickcheck_bamfiles.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

samtools quickcheck -vvv "${mappedfolder}"*_Aligned.sortedByCoord.out.bam \
            &>> ${logfile}
echo "operation ends at $(date)" >> ${logfile}            

# to print user's message (samtools quickcheck is really quick!) and record it
samtools quickcheck "${mappedfolder}"*_Aligned.sortedByCoord.out.bam \
            && inbrief="All files are goods." \
            || inbrief="At least one file is corrupted, see logfile."
echo ${inbrief} | tee -a ${logfile}

<div class="alert alert-block alert-warning">
    <b>If there is an issue with your files</b>, you should start again previous steps for corrupted file(s). Indeed, some downstream tools may work but <b><i>samtools</i></b> will fail. Besides, you wouldn't be confident with generated results: <b>save time, take some before losing later!</b>
</div>

---
---
## 2 - Some metrics with <code>samtools</code>
---

The commands used for this part belong to a large package of utilities that are very useful to manage those types of files: **Samtools** (http://www.htslib.org/).

### 2.1 - Preparing command line variables
---

An option would be to save results files along with `.bam` files but it's not the best idea to have a lot of files in one folder (especially when saving them, along with long list handling). So we will save the output files to another folder.

In [ ]:
## Code cell 12 ##

samtoolsfolder="${gohome}$USER/Results/samtools/"

mkdir -p ${samtoolsfolder}

Count files are *per se* the interesting outputs but, if there is any issue or for duration information, we nonetheless keep some details into a `.log` file saved with the others reports.

In [ ]:
## Code cell 13 ##

logfolder="${gohome}$USER/Results/logfiles/"

--
### 2.2 - Running command line for <code>samtools</code> metrics
---

We are using two tools as part of samtools to generate metrics on bam files:
- `stats` http://www.htslib.org/doc/samtools-stats.html  produces comprehensive statistics from alignment file 
- `flagstat` http://www.htslib.org/doc/samtools-flagstat.html counts the number of alignments for each FLAG type 

<div class="alert alert-block alert-warning"><b>Warning:</b>If you did not use previous notebook P04 of this pipeline, please remove or adapt <code>_sortedByCoord.out</code> in the fifth line below in the loop, in order that filename pattern suits your filename format.</b></div>


In the loop below, we select input bam files with a `_sortedByCoord.out.bam` suffix.

>Note that STAR can produce another bam file with the suffix `_Aligned.toTranscriptome.out.bam` if you used the *quantification mode* with the option `TranscriptomeSAM` to get transcripts counts. Such .bam files are not standard ones as reads are sorted by transcript name instead of chromosome position.




In [ ]:
## Code cell 14 ##

logfile="${logfolder}samtools_stats_flagstat_bamfiles.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

time for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do

    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "======= Processing sampleID: ${id}..." | tee -a ${logfile}

    statsfile="${samtoolsfolder}${id}.stats"
    echo "$(date) starting ${statsfile}" >> ${logfile}
    samtools stats ${fn} > ${statsfile}

    flagsfile="${samtoolsfolder}${id}.flagstat"
    echo "$(date) starting ${flagsfile}" >> ${logfile}
    samtools flagstat ${fn} > ${flagsfile}
    
    echo "... done" | tee -a ${logfile}

done

echo "operation ends at $(date)" >> ${logfile}

echo "=== files created after samtools counts ===" >> ${logfile}
ls -lh "${samtoolsfolder}" >> ${logfile}

echo "There are $(ls "${samtoolsfolder}" | wc -l) output files."

---
---
## 3 - Some metrics and graphical results with Qualimap
---

### 3.1 - Tool introduction and preparation step
---

We will run the **QUALIMAP** program (http://qualimap.conesalab.org/), that collects the data about the `.bam file`, including coverage estimation and many other parameters, and reports a summary of the main properties of the alignment data.

Qualimap reads `.sorted.bam` files and generates a folder containing a report on `.html` format.

As you can see from the next command to know which version we are running, Qualimap includes several tools. We will be using the classical `bamqc tool` which can work on any kind of NGS .bam files. We will also use `rnaqc tool` which is dedicated to RNASeq.

In [ ]:
## Code cell 15 ##

qualimap --help

> Remark: QUALIMAP can also be launched on most OS using a command line that will open a user-friendly Java window. On Mac or Windows, you may have to modify the memory RAMs to run it. It is all explained in the documentation.

As with previous tools, we will put its results in a `Results/` subfolder:

In [ ]:
## Code cell 16 ##

qualimapfolder="${gohome}$USER/Results/qualimap/"
mkdir -p ${qualimapfolder}

An issue with current Qualimap releases is that we can't change file names for both tools we will use.  
Thus, let's create distinct folders for the two tools within the qualimap folder.

In [ ]:
## Code cell 17 ##

bamqcfolder="${qualimapfolder}bamqc/"
mkdir -p ${bamqcfolder}

In [ ]:
## Code cell 18 ##

rnaseqfolder="${qualimapfolder}rnaseq/"
mkdir -p ${rnaseqfolder}

--
### 3.2 - General view with <code>qualimap bamqc</code>
---

The **qualimap** options we will use are: <br>

- `-bam filepath`: input mapping file in BAM format, with reads sorted by coordinates
- `-gff filepath` or `--feature-file path/to/file`, the path to the feature file with regions of interest in GFF/GTF or BED format
- `-outdir text`: output folder for HTML report and raw results.
- `-p VALUE` or `--sequencing-protocol VALUE`, sequencing library protocol: strand-specific-forward, strand-specific-reverse or non-strand-specific (default).
- `c` or `--paint-chromosome-limits`: paint chromosome limits inside charts
- `outformat`: Qualimap produces by default an HTML report but we may ask for a PDF or both using corresponding predefined values (PDF, HTML or PDF:HTML for both).

<ul class="alert alert-block alert-info">
    <li>
        See <a href="http://qualimap.conesalab.org/doc_html/analysis.html#bam-qc">online manual</a> for more details
    </li>
</ul>

<div class="alert alert-block alert-danger"> <a id="section3.2"></a>
    Following <b>command is prepared for usage on a computational cluster</b> such as the <i>Institut Français de Bioinformatique</i> (IFB)'s core cluster. We use a session defined as <b>5 CPU with 21 GB available for RAM</b>. Adapt it to your ressources.
</div>

In [ ]:
## Code cell 19 ##

logfile="${logfolder}qualimap_bamqc_3samples.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do   
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}

    myoutdir="${bamqcfolder}${id}"
    echo "destination folder is ${myoutdir}" >> ${logfile}
    
    echo "qualimap bamqc starts at $(date)" >> ${logfile}
    qualimap bamqc -bam ${fn} \
                    --feature-file "${gtffile}" \
                    --sequencing-protocol non-strand-specific \
                    --paint-chromosome-limits \
                    -outdir ${myoutdir} \
                    --java-mem-size=20G \
                    &>> ${logfile}
    echo "qualimap bamqc ends at $(date)" >> ${logfile}
    
    echo "... done" | tee -a ${logfile}
  
done 

In [ ]:
## Code cell 20 ##
echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after qualimap bamqc ===" >> ${logfile}
tree "${bamqcfolder}" >> ${logfile}
echo "Qualimap generated $(find "${bamqcfolder}" -name *.html | wc -l) html reports."

We can know how long this cell lasts, thanks to ``logfile``:

In [ ]:
## Code cell 21 ##

grep -e "operation" ${logfile}

--
### 3.3 - Specific RNAseq plots with <code>qualimap rnaseq</code>
---

``qualimap rnaseq``, a Qualimap specific option for RNAseq data, gives some more metrics: 
- a *Reads genomic origin* circus-plot to check that reads map to exons
- two *Coverage Profile Along Genes* plots (low and high)
- a *Junction Analysis* circus-plot to distinguish between known and novel junctions

<ul class="alert alert-block alert-info">
    <li>
        See <a href="http://qualimap.conesalab.org/doc_html/analysis.html#rna-seq-qc">online manual for this tool</a> for more details
    </li>
</ul>

#### **3.3.a - Understanding operations**

With paired end data, we could use following lines:  
<code>qualimap rnaseq -pe \  
               -bam path/to/sortedByNames.bam \ 
               -gtf path/to/gencode_annotation.gtf \ 
               -outdir path/to/outputfolder/</code>

Those options, along with others, are explained in manual:
<blockquote>
     <code>-pe</code> or <code>--paired</code> Setting this flag for paired-end experiments will result in counting fragments instead of reads <br>
     <code>-bam path/to/file.bam</code>, to locate input mapping file in BAM format. <br>
     <code>-gtf path/to/file.gtf</code>, where to find annotations file in Ensembl GTF format (no compressed format). <br>
     <code>-outdir path/to/folder/</code> to define output folder for HTML report and raw data. <br> 
</blockquote>

There are some more options of interest and we will use the first two while letting the third one by default:
<blockquote>
     <code>-p VALUE</code> or <code>--sequencing-protocol VALUE</code>, to precise sequencing library protocol. VALUES among: strand-specific-forward, strand-specific-reverse or non-strand-specific (default) <br>
     <code>-oc path/to/outfile.counts</code>, to place output file for computed counts in a distinct folder. If only name of the file is provided, then the file will be saved in the output folder. <br>
     <code>-a</code> or <code>--algorithm</code>, to set counting algorithm: uniquely-mapped-reads(default) or proportional.
</blockquote>

Contrary to `bamqc`, `rnaseq` Qualimap tool needs alignment files to be sorted by read names. By default, if given otherwise, it will sort `.bam` file... but we already noticed that handling those big files is quite slow with only one process running at a time.  
So we will use `samtools sort` tool to perform it with multiple threads at a time and give `qualimap rnaseq` an already sorted input file and let it know with <code>-s</code> or <code>--sorted</code> (Only required for paired-end analysis...).

In order to follow this process, we will use this command format:  
<code>samtools sort -n \ 
            -@ X -m YG \ 
            -O BAM \ 
            -o path/to/sortedByNames.bam \ 
            -T path/to/temporaryfolder/ \ 
            path/to/input.bam</code>

where options correspond to:  
<blockquote>
    <code>-n</code> for activating sorting by read name (<i>i.e.</i> the QNAME field) <br>
    <code>-o FILE</code>    Write final output to FILE rather than standard output <br>
    <code>-T PREFIX</code>, to write temporary files to a particular folder. Used names are <i>PREFIX.nnnn.bam</i>, where <i>nnnn</i> stands for a 4-digit number <br>
    <code>-O</code> or <code>--output-fmt FORMAT</code> to specify output format (SAM, BAM or CRAM). If not mentionned, <code>samtools</code> will deduce it from file extension (not available if there is no written output file!) <br>
    <code>-@ INT</code> or <code>--threads INT</code>, the number of additional threads to use [0]. Indeed, you will  only use 1 thread by default. <br>
    <code>-m INT</code>, to limit maximum memory per thread; suffix K/M/G recognized [768M]
</blockquote>

#### **3.3.b - Running <code>qualimap rnaseq</code> loop... with patience**

<div class="alert alert-block alert-warning">
    Below cell will run for <b>at least 30 minutes per sample</b>. Either you come back later or you may use JupyterLab option to <i>Run selected cell and all below</i> in <i>Run</i> Menu (top bar).
</div>

<div class="alert alert-block alert-danger"> <a id="section3.2"></a>
    Following <b>command is prepared for usage on a computational cluster</b> such as the <i>Institut Français de Bioinformatique</i> (IFB)'s core cluster. We use a session defined as <b>5 CPU with 21 GB available for RAM</b>. Adapt it to your ressources.
</div>

In [ ]:
## Code cell 22 ##

logfile="${logfolder}qualimap_rnaseq_3samples.log"

In [ ]:
## Code cell 23 ##

echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do  
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}
    echo "input filename: ${fn}"
    
    # some preparation
    mytempfile="${rnaseqfolder}${id}_Aligned.sortedByNames.bam"
    myoutdir="${rnaseqfolder}${id}/"
    mkdir -p ${myoutdir}
    echo "destination folder ${myoutdir}" >> ${logfile}
    
    # bam sorting...
    echo "samtools starts at $(date)" >> ${logfile}
    
    samtools sort -n \
                --threads 3 -m 5G \
                --output-fmt BAM \
                -o ${mytempfile} \
                -T ${rnaseqfolder} \
                --verbosity 3 \
                ${fn} \
                &>> ${logfile}
    
    echo "samtools ends at $(date)" >> ${logfile}

    # some user conversation to help being patient

    echo "...changing tool..." | tee -a ${logfile}

    # then rnaseq QC from Qualimap
    echo "qualimap rnaseq starts at $(date)" >> ${logfile}
    qualimap rnaseq -bam ${mytempfile} \
                        --sorted \
                        -gtf "${gtffile}" \
                        --sequencing-protocol non-strand-specific \
                        --paired \
                        -outdir ${myoutdir} \
                        --java-mem-size=20G \
                        &>> ${logfile}
    echo "qualimap rnaseq ends at $(date)" >> ${logfile}
    
    # removing extra bam file... saving disk space
    rm ${mytempfile}
    
    echo "... done" | tee -a ${logfile}
    
done

In [ ]:
## Code cell 24 ##
echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after qualimap rnaseq ===" >> ${logfile}
tree "${rnaseqfolder}" >> ${logfile}
echo "Qualimap generated $(find "${rnaseqfolder}" -name *.html | wc -l) html reports." | tee -a ${logfile}

In [ ]:
## Code cell 25 ##

#logfile="/shared/projects/2312_rnaseq_cea/scaburet/Results/logfiles/qualimap_rnaseq_3samples.log"
grep -e "operation" ${logfile}

---
---
## 4- Having a summary report with MultiQC
---

### 4.1 - Preparations steps
---

#### **4.1.a - Tool presentation and version**

When numerous samples are processed, it can easily become tedious to look in each mapping quality report. So we'll run MultiQC (https://multiqc.info/), that scans automatically a folder for all quality checks outputs and produce a single report. MutliQC runs on almost any possible NGS tools (https://multiqc.info/docs/#multiqc-modules).

In [ ]:
## Code cell 26 ##

multiqc --version

#### **4.1.b - Personnalize the report: setting variables**

Please, specify **file name** you want to have (do not worry about extension, MultiQC will handle this for us) inside quotes in the next cell. <a id="multiqctextvar"></a>  
<b>DO NOT use spaces or any special characters!</b>

In [ ]:
## Code cell 27 ##

inamemyfile="4_starAlignments-bam-files-3samples"

Please, specify a meaningful **title** inside quotes in the next cell, to display at the head of your oncoming report.
<b>No more space limit but still avoid any special characters.</b> 

In [ ]:
## Code cell 28 ##

mytitle="Sorted by coordinates bam files quality for 3 paired end sequenced samples, from double-Htz B-ALL mice study (Dataset GSE158673, subSerie GSE158661)"

Besides, we can add a comment in header's report. It's a good practise to do so. So we will define it in following cell.

> In this cell, we use several lines to keep it readable when displaying notebook. As your text lines are just collapsed together in the html report, be sure to keep last blank space when ending every line).

In [ ]:
## Code cell 29 ##

mycomment=$(echo "Bam files derived from bulk RNA sequencing (mouse, unstranded) " \
"Performed by Ramamoorthy et al. 2020 (PMID:  33004416 ; GEO: GSE158673, subSerie GSE158661 ; SRA: PRJNA666155). " \
"RNASeq analysis to unravel molecular networks driving leukemia in Ebf1+/-Pax5+/- (dHet) B-ALL mice : To profile gene expression changes in  Ebf1+/-Pax5+/- (dHet) leukemic mice, RNASeq analysis was performed in dHet B-ALL, dHet proB and wt proB
cells.     " \
"PRJNA666155: 3 dHet mice, 2 replicates; proB cells derived from dHet pre-leukemic mouse, 3 replicates; wt proB cells, 2 replicates. " \
"Mapping to the reference genome (mouse, Gencode GRCm39, release 32) performed by STAR. Additionnal settings:  " \
"Unmapped or too multimapped (>20) are kept in separate file, GeneCounts quantification mode activated.")

... and don't forget to specify the output folder!

In [ ]:
## Code cell 30 ##

qcsummaries="${gohome}$USER/Results/multiqc/"

--
### 4.2 - Generate MultiQC summary report
---

MultiQC is verbose but, as it will work only on FastQC reports, it is quite short.  *This time, we will have qualimap bamqc, qualimap rnaseq, samtools stats and samtools idxstats*!
So, we will let it lines show in notebook while saving them in a file for later use.

In [ ]:
## Code cell 31 ##

logfile="${logfolder}multiqc-processing_mapped-quality-3samples.log"
echo "Screen output is also saved in ${logfile}"

# as time command does not redirect output
echo "operation starting at $(date)" >> ${logfile}
time multiqc --interactive --export \
        --module star ${mappedfolder} \
        --module samtools ${samtoolsfolder} \
        --module qualimap ${qualimapfolder} \
        --outdir "${qcsummaries}" \
        --filename "${inamemyfile}" \
        --title "${mytitle}"  \
        --comment "${mycomment}" \
        "${gohome}${USER}/Results/" \
        |& tee -a ${logfile}
echo "operation finished at $(date)" >> ${logfile}

# to see which files we have afterward and follow folder sizes
ls -lh "${qcsummaries}" >> ${logfile}

du -h -d1 "${gohome}$USER/Results/" >> ${logfile}

--
### 4.3 - Retrieve MultiQC report for all samples
---

Similar reports were generated for all 11 samples. They are available in the allData folder, and we can have a copy in our personal folder for later use.

In [ ]:
## Code cell 32 ##   

cp -r "${gohome}allData/Results/multiqc/4_starAlignments-bam-files_data" "${gohome}$USER/Results/multiqc/"
cp -r "${gohome}allData/Results/multiqc/4_starAlignments-bam-files_plots" "${gohome}$USER/Results/multiqc/"
cp "${gohome}allData/Results/multiqc/4_starAlignments-bam-files.html" "${gohome}$USER/Results/multiqc/"

---
---
## 5 - Monitoring disk space usage
---

In [ ]:
## Code cell 33 ##

du -h -d2 ${gohome}$USER

---
___

## Conclusion


**Next Practical session**

Now we go on to reads quantification at gene level.  
  
**=> Step 6: Reads Count** 

The jupyter notebook used for the next session will be *Pipe_06-bash_reads-counts.ipynb.ipynb*    
Let's retrieve it in our directory, in order to have a private copy to work on:   

In [ ]:
## Code cell 34 ##   

cp "${gohome}pipeline/Pipe_06-bash_reads-counts.ipynb" "${gohome}$USER/"



**Save executed notebook**

To end the session, save your exectued notebook in your `run_notebooks' folder. Adjust the name with yours and reformat as code cell to run it.

<div class="alert alert-block alert-success"><b>Success:</b> Well done! You now know how to do QCs on mapped reads.<br>
Don't forget to save you notebook and export a copy as an <b>html</b> file as well <br>
- Open "File" in the Menu<br>
- Select "Export Notebook As"<br>
- Export notebook as HTML<br>
- You can then open it in your browser even without being connected to the server! 
</div>

---
---

## Useful commands
<div class="alert alert-block alert-info"> 
    
- <kbd>CTRL</kbd>+<kbd>S</kbd> : save notebook<br>    
- <kbd>CTRL</kbd>+<kbd>ENTER</kbd> : Run Cell<br>  
- <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> : Run Cell and Select Next<br>   
- <kbd>ALT</kbd>+<kbd>ENTER</kbd> : Run Cell and Insert Below<br>   
- <kbd>ESC</kbd>+<kbd>y</kbd> : Change to *Code* Cell Type<br>  
- <kbd>ESC</kbd>+<kbd>m</kbd> : Change to *Markdown* Cell Type<br> 
- <kbd>ESC</kbd>+<kbd>r</kbd> : Change to *Raw* Cell Type<br>    
- <kbd>ESC</kbd>+<kbd>a</kbd> : Create Cell Above<br> 
- <kbd>ESC</kbd>+<kbd>b</kbd> : Create Cell Below<br> 

<em>  
To make nice html reports with markdown: <a href="https://dillinger.io/" title="dillinger.io">html visualization tool 1</a> or <a href="https://stackedit.io/app#" title="stackedit.io">html visualization tool 2</a>, <a href="https://www.tablesgenerator.com/markdown_tables" title="tablesgenerator.com">to draw nice tables</a>, and the <a href="https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd" title="Ultimate guide">Ultimate guide</a>. <br>
Further reading on JupyterLab notebooks: <a href="https://jupyterlab.readthedocs.io/en/latest/user/notebook.html" title="Jupyter Lab">Jupyter Lab documentation</a>.<br>   
</em>    
 
</div>

---
Bénédicte Noblet - 05-07 2021   
Sandrine Caburet - 02-05 2023   
Claire Vandiedonck - 01-06 2023  
Maj 03/06/2023 by @CVandiedonck